In [13]:
# !/usr/bin/python
# -*- coding: utf-8 -*-
# Author: Selvaria
# 完全使用selenium获取企业信息存入dic_company_detail以及dic_company表

import requests
import time
from bs4 import BeautifulSoup
import csv
import pandas
import random
from selenium import webdriver
import json
import pymysql


  
def get_login(): #利用selenium手动登录,获取cookies
    url = 'https://www.qichacha.com/user_login'
#         url = 'https://www.qichacha.com'
    browser = webdriver.Chrome() 
    browser.get(url) 
    #login_click = browser.find_element_by_xpath('/html/body/header/div/ul[2]/li[7]/a')  #登陆
    #browser.execute_script("arguments[0].click();", login_click)
    time.sleep(40) #登录并录入验证码  #15201760096 123456  #18698925437 #13342264161 
    req = requests.Session()
    cookies = browser.get_cookies()
    
    req = requests.Session()
    for cookie in cookies:
        req.cookies.set(cookie['name'],cookie['value'])
    return cookies

cookie = get_login()
print(type(cookie))
data = json.dumps(cookie)
with open('qichacha_cookies.txt', 'w') as f:    
    f.write(data)
print(data)

<class 'list'>
[{"domain": "www.qcc.com", "expiry": 1584102471, "httpOnly": false, "name": "hasShow", "path": "/", "secure": false, "value": "1"}, {"domain": ".qcc.com", "expiry": 1615595271, "httpOnly": false, "name": "zg_de1d1a35bfa24ce29bbf2c7eb17e6c4f", "path": "/", "secure": false, "value": "%7B%22sid%22%3A%201584059254132%2C%22updated%22%3A%201584059271565%2C%22info%22%3A%201584059254141%2C%22superProperty%22%3A%20%22%7B%7D%22%2C%22platform%22%3A%20%22%7B%7D%22%2C%22utm%22%3A%20%22%7B%7D%22%2C%22referrerDomain%22%3A%20%22%22%2C%22cuid%22%3A%20%2261eb3dde4c5f40a8c979c5fd3f65efc5%22%7D"}, {"domain": "www.qcc.com", "expiry": 1899419254, "httpOnly": false, "name": "_uab_collina", "path": "/", "secure": false, "value": "158405925439096253271956"}, {"domain": "www.qcc.com", "expiry": 1599784071, "httpOnly": false, "name": "CNZZDATA1254842228", "path": "/", "secure": false, "value": "651271944-1584054777-%7C1584054777"}, {"domain": ".qcc.com", "expiry": 1584664071.071493, "httpOnly": fa

In [2]:
#去除企查查无用企业
err_l = []
with open(r'企查查无结果企业.txt','r',encoding='utf8') as r:
    rows=r.readlines()
for l in rows:
    err_l.append(l.split('\n')[0])

# 注意修改文件路径分别补全发票表和交易表
with open(r'D:\Selvaria\scripts\company\changxing_invoice\files\nocom190510.txt','r',encoding='utf8') as r:
# with open(r'D:\Selvaria\scripts\company\changxing_invoice\files\billnocom191203.txt','r',encoding='utf8') as r:
    lines=r.readlines() 
#     print(lines)
with open(r'需录入企查查企业_test.txt','w',encoding='utf8') as w:
    for l in lines:
        if l.split('\n')[0] not in err_l:
            w.write(l)

In [ ]:
#主程序
import requests
import time
from bs4 import BeautifulSoup
import csv
import pandas
import random
from selenium import webdriver
import json
import pymysql

# connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')


class QichachaSelenium(object):

    def get_cookies(self):
        f= open('qichacha_cookies.txt')
        data_json = f.read()
        data_handle = json.loads(data_json)
        return data_handle

    def get_company_info(self, name, cookies): 
        #name = '金钻石油机械股份有限公司'

        url_search = 'https://www.qcc.com/search?key=%s' %name #目标网址
        browser = webdriver.Chrome()
        url_l= 'https://www.qcc.com/user_login' #这一步特别重要，必须先打开一个目标网址对应的界面
        browser.get(url_l) 
        for cookie_item in cookies:
            #print({'name': cookie_item, 'value': cookies[cookie_item]})
            browser.add_cookie(cookie_item)
        browser.get(url_search)

        try:
            num = browser.find_element_by_xpath('//*[@id="countOld"]/span').text
        except Exception as e:
            print('基本被封，请手动查询')
            print(str(e))
            #return None

        if num.strip() == '0':
            print('无此企业', name)
            return 0
        else:
            time.sleep(1)
            search_url = browser.find_element_by_xpath('//*[@id="search-result"]/tr[1]/td[3]/a').get_attribute("href") 
            print(search_url)
#             url_c = 'https://www.qcc.com%s' %'/cbase'+search_url[29:-5]
#             url_c = 'https://www.qcc.com/'+search_url
            url_c = search_url
#             print(url_c)
            #com_web_q = 'https://www.qichacha.com' + search_url #企查查的企业主页
            url_company = browser.get(url_c) 
            time.sleep(1)
            button = browser.find_element_by_xpath('//*[@id="firstepdadModal"]/div/div/div[2]/button')
            browser.execute_script("arguments[0].click();", button)
            try:
                time.sleep(1)
                #table =  browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody').text
                #print(table)
                company_title = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[1]/h1').text #企业名
                company_lead = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[2]/div/div/div[2]/a[1]/h2').text #法人
                company_tel = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[3]/div[1]/span[1]/span[2]').text.strip()
                company_email = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[3]/div[2]/span[1]/span[2]').text.strip()\
                .split(' ')[0]
                #print(company_lead, company_tel, company_email)

                found_cash = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[2]/td[2]').text.strip() #注册资本
                found_date = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[6]').text.strip() #成立日期
                company_loc = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[8]/td[2]').text.split('\n')[0].split(' ')[0] #企业地址'
                company_cover = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[9]/td[2]').text.strip() # 经营范围
                company_tax_num = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[4]/td[2]').text.strip() #'纳税人识别号'
                company_prov = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[6]/td[6]').text.strip()  #所属地区
                #print('公司名称：%s' %company_title)
                try:
                    info_0 = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[4]').text.strip() #经营状态
                except:
                    info_0 = None
                if '在业' in info_0:
                    info = 1
                elif '存续' in info_0:
                    info = 2
                elif '注销' in info_0:
                    info = 10
                elif '吊销' in info_0:
                    info = 9
                elif '筹建' in info_0:
                    info = 3
                elif '清算' in info_0:
                    info = 4
                elif '迁入' in info_0:
                    info = 5
                elif '迁出' in info_0:
                    info = 6
                elif '停业' in info_0:
                    info = 7
                elif '撤销' in info_0:
                    info = 8
                elif '开业' in info_0:
                    info = 1
                else:
                    info = None
                
                #如果使用会计原名，则使用name字段，如果使用网上注册的标准名，使用company_title字段
                msg = [company_prov, name, company_lead, found_date, found_cash, company_loc, company_email, company_tel,\
                       company_cover, search_url, company_tax_num, info]
                print('%s company is done.' %name)
                #print(msg)
                browser.quit()
                return msg
            except Exception as e:
                print(repr(e))
                print('企业网页数据提取出现问题')
                return 0

    def up_sql(self, data): #上传数据库dic_company_detail
        '''
        '公司名', '法人', '成立日期', '注册资本', '地址', '邮箱', '电话', '经营范围', '企查查主页', '纳税识别号', '公司id'
        '''
        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
#         connection = connection
        try :
            with connection.cursor() as cursor:
                sql_i = "insert into `dic_company_detail`(`name`,`person`,`date`,`capital`,`address`,`email`,`phone`,`operate`,`website`,\
                `tax_num`,`company_id`)\
                values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(sql_i,data)
            connection.commit() 
            #print('成功')
        except Exception as e:
            print(e)
        finally:
            connection.close()
        print('"dic_company_detail" Single is Done')

    def data_handle_txt(self, msg, name): #txt数据整理排序
        msg = msg[:-1] #去掉经营状态
        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
#         connection = connection
        sql_up = "select id from dic_company where name = '%s'"
        cursor = connection.cursor()
        cursor.execute(sql_up % name)
        c_id = cursor.fetchone()[0]
        msg = msg + [c_id,]
        #print(len(msg[1:]))
        return msg[1:]

    def get_txt(self,path,encoding='utf8'): #待更新数据从文本txt获取
        f = open(path,encoding=encoding)
        data_name = f.read()
        data_list = data_name.split('\n')
        return data_list

    def insert_c_sql(self, data): #新企业上传数据库dic_company
        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
#         connection = connection
        if '长兴岛' in data[5]:
            try :
                with connection.cursor() as cursor:
                    sql_i = "INSERT INTO `dic_company` (`name`,`com_group`,`com_type`,`company_province`,`company_provinceId`,`company_city`,`company_cityId`)\
                    VALUES (%s,%s,%s,%s,%s,%s,%s)"
                    cursor.execute(sql_i,[data[1],1,data[-1],'辽宁省',8,'大连市',51])
                connection.commit() 
                #print('成功')
            except Exception as e:
                print(e)
            finally:
                connection.close()
        else:
            try :
                with connection.cursor() as cursor:
                    sql_i = "INSERT INTO `dic_company` (`name`,`com_group`,`com_type`,`company_province`)\
                    VALUES (%s,%s,%s,%s)"
                    cursor.execute(sql_i,[data[1],0,data[-1],data[0]])
                connection.commit() 
                #print('成功')
            except Exception as e:
                print(e)
            finally:
                connection.close()
        print('"dic_company" Single is Done')
    

            
def main_txt(): #新增用
    q = QichachaSelenium()
    cookies = q.get_cookies()
    data = q.get_txt('需录入企查查企业_test.txt')
#     data = ['大连大昌石油化工有限公司', '大连凯昌石油化工有限公司', '中油天禹（大连）石化有限公司', '大连鸿利达新材料有限公司']
    error_list = []
#     for index in range(7660,len(data)): 
    print(len(data)) # 3636
    for index in range(950,len(data)):
        print('now index is:',index)        
        name = data[index]
        if name == '':
            print('txt空值')
            continue
        elif len(name) < 5:
            print(name)
            print('无效企业')
            continue
        elif '科目余额表' in name:
            print(name)
            print('无效企业')
            continue
        elif '序时账' in name:
            print(name)
            print('无效企业')
            continue
        
        try:
            msg = q.get_company_info(name, cookies)
            print(msg)
            if msg == 0:
                error_list.append(name)
                time.sleep(random.uniform(1.4,4.4))
                print('已添加到不符企业名称')
                print(error_list)
                time.sleep(random.uniform(44,55.7))
                continue
            q.insert_c_sql(msg) #上传数据库dic_company
        except pymysql.err.IntegrityError:
            data_up = q.data_handle_txt(msg, name)
            #print(data_up)
            q.up_sql(data_up)
            #break
            continue
        try:
            data_up = q.data_handle_txt(msg, msg[1]) #这里要使用之前查的名字，因为不一定能在企查查上查到
            #print(data_up)
#             break            
            q.up_sql(data_up) #上传数据库dic_company_detail
            time.sleep(random.uniform(54,147.7))
        except Exception as e:
            print(str(e))
            print(name,'公司数据处理出现问题')
            error_list.append(name)
            print(error_list)
            return error_list
            break
        
        
        
if __name__ == '__main__':
    e_list = main_txt()
    print(e_list)

3636
now index is: 864
https://www.qcc.com/firm_b6e23607a7808d215d36ae57023c160d.html
多维联合集团有限公司 company is done.
['', '多维联合集团有限公司', '', '', '', '', 'lyq@duowei.net.cn', '010-56305030 更多号码', '', 'https://www.qcc.com/firm_b6e23607a7808d215d36ae57023c160d.html', '', None]
"dic_company" Single is Done


D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'date' at row 1")
  result = self._query(query)


"dic_company_detail" Single is Done
now index is: 865
https://www.qcc.com/firm_ac81e6bd34646bbd7f542136b041c63a.html
北京星之烨工程机械设备租赁有限公司 company is done.
['北京市', '北京星之烨工程机械设备租赁有限公司', '黄河', '2018-04-04', '1000万元人民币', '北京市怀柔区九渡河镇黄坎村735号', '18321766675@163.com', '18321766675', '租赁建筑工程机械、建筑工程设备;工程项目管理;技术开发、技术转让、技术咨询、技术交流、技术推广;施工总承包;专业承包;劳务分包;设计、制作、代理、发布广告;销售五金交电(不含电动自行车)。(企业依法自主选择经营项目,开展经营活动;依法须经批准的项目,经相关部门批准后依批准的内容开展经营活动;不得从事本市产业政策禁止和限制类项目的经营活动。)', 'https://www.qcc.com/firm_ac81e6bd34646bbd7f542136b041c63a.html', '91110116MA01B8122C', 1]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 866
https://www.qcc.com/firm_80217cd940b488521e34c16761970f84.html
四川河翔水利水电工程有限责任公司 company is done.
['', '四川河翔水利水电工程有限责任公司', '', '', '', '', '1057284206@qq.com', '028-87784943 更多号码', '', 'https://www.qcc.com/firm_80217cd940b488521e34c16761970f84.html', '', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 867
https://www.qcc.com/firm_dc13c1e06adbb

D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'operate' at row 1")
  result = self._query(query)


"dic_company_detail" Single is Done
now index is: 883
https://www.qcc.com/firm_e1b12204616cb18feb3896ea4f5186bc.html
唐山市标达沃业石油产品有限公司 company is done.
['河北省', '唐山市标达沃业石油产品有限公司', '李强', '2017-07-24', '3000万元人民币', '唐山市路北区福地大厦806', '1297555654@qq.com', '0315-2065630 更多号码', '汽油、煤油、柴油(闭杯闪点≤60℃)、燃料油、混合芳烃、石脑油、石油原油、溶剂油(闭杯闪点≤60℃)、甲基叔丁基醚、煤焦油、粗苯、苯、1.2-二甲苯、1.3-二甲苯、1.4-二甲苯煤、煤焦沥青(无存储)批发;水泥制品、玻璃制品、塑料制品、建材、五金产品、润滑油、专用化学产品、合成纤维、橡胶制品、化学合成材料批发零售***(依法须经批准的项目,经相关部门批准后方可开展经营活动)', 'https://www.qcc.com/firm_e1b12204616cb18feb3896ea4f5186bc.html', '91130203MA08U3WP4X', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 884
https://www.qcc.com/firm_ff7bfe00fd9d426bc9c9520d582bd9cd.html
青海振通建筑基础有限公司 company is done.
['', '青海振通建筑基础有限公司', '', '', '', '', '1093556106@qq.com', '0971-8822707 更多号码', '', 'https://www.qcc.com/firm_ff7bfe00fd9d426bc9c9520d582bd9cd.html', '', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 885
https://www.qcc.com/firm_7095b36

...

"dic_company_detail" Single is Done
now index is: 977
https://www.qcc.com/firm_73b9eda85cc5b238ff350e71c462042e.html
汤原县忠石加油站 company is done.
['黑龙江省', '汤原县忠石加油站', '何居宜', '2018-12-17', '-', '黑龙江省佳木斯市汤原县香兰镇哈肇公路洪胜村对面(香兰镇铁路北侧京抚路西侧)', '7177054@qq.com', '18545028555', '汽油、柴油零售;食品零售。', 'https://www.qcc.com/firm_73b9eda85cc5b238ff350e71c462042e.html', '91230828MA1BDW1C13', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 978
https://www.qcc.com/firm_2c36a12c7e8f6624c1abe39babef267d.html
集贤县中石成品油经销有限公司 company is done.
['黑龙江省', '集贤县中石成品油经销有限公司', '蔡勇杰', '2016-07-06', '80万元人民币', '黑龙江省双鸭山市集贤县福利镇西环路东侧四达国际斜对面', '1115353473@qq.com', '15045723235', '零售:汽油、柴油、润滑油、预包装食品;便利店零售服务。', 'https://www.qcc.com/firm_2c36a12c7e8f6624c1abe39babef267d.html', '91230521MA18YDX90N', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 979
https://www.qcc.com/firm_100e0d1a11c0f02b9735e68bfc964a98.html
木兰县柳河石油站 company is done.
['黑龙江省', '木兰县柳河石油站', '韩淑琴', '20

"dic_company_detail" Single is Done
now index is: 996
https://www.qcc.com/firm_70c20b634b9713383ea11400db0912e2.html
迁西县上营乡瓦房庄村印福加油站 company is done.
['河北省', '迁西县上营乡瓦房庄村印福加油站', '冯莉', '2002-04-29', '-', '迁西县瓦房庄村', '暂无', '0315-5620222 更多号码', '润滑油、汽油、柴油(闭杯闪点≤60℃)零售(依法须经批准的项目,经相关部门批准后方可开展经营活动)**', 'https://www.qcc.com/firm_70c20b634b9713383ea11400db0912e2.html', '91130227738705146Y', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 997
https://www.qcc.com/firm_8c0d6dade0ddc5a6304393200cf7249d.html
宽城双利精选有限公司 company is done.
['', '宽城双利精选有限公司', '', '', '', '', '417366643@qq.com', '0314-6987518 更多号码', '', 'https://www.qcc.com/firm_8c0d6dade0ddc5a6304393200cf7249d.html', '', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 998
https://www.qcc.com/firm_9a8ad6c2f18c72e33bf77c57e5e9106b.html
青龙满族自治县顺龙货物运输有限公司 company is done.
['河北省', '青龙满族自治县顺龙货物运输有限公司', '张峰', '2018-05-23', '100万元人民币', '河北省秦皇岛市青龙满族自治县双山子镇沟口子村', '暂无', '13930383621'

"dic_company_detail" Single is Done
now index is: 1015
https://www.qcc.com/firm_337aac76aaf9f530c022b9fa8e59a67f.html
宽城隆丰矿业有限公司 company is done.
['', '宽城隆丰矿业有限公司', '', '', '', '', '暂无', '0314-6967398', '', 'https://www.qcc.com/firm_337aac76aaf9f530c022b9fa8e59a67f.html', '', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 1016
https://www.qcc.com/firm_6324bfd046bc3603bb794623b894bb1d.html
卢龙腾澳混凝土有限公司 company is done.
['河北省', '卢龙腾澳混凝土有限公司', '双旭', '2018-09-04', '2600万元人民币', '河北省秦皇岛市卢龙县城永平大街西段北侧', '暂无', '18733492211', '商用混凝土、沥青混凝土、预拌砂浆、水泥稳定土的制造、销售**(依法须经批准的项目,经相关部门批准后方可开展经营活动)', 'https://www.qcc.com/firm_6324bfd046bc3603bb794623b894bb1d.html', '91130324MA0CP2JR1L', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 1017
https://www.qcc.com/firm_302fdbd45cd1d9edd35d691af742b731.html
宽城双博矿业有限公司 company is done.
['', '宽城双博矿业有限公司', '', '', '', '', '暂无', '13031416748 更多号码', '', 'https://www.qcc.com/firm_302fdbd45cd1d9edd35d691af

"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 1035
https://www.qcc.com/firm_941ef081134195e915552606aef17c0d.html
秦皇岛峻森土石方工程有限公司 company is done.
['河北省', '秦皇岛峻森土石方工程有限公司', '杨长喜', '2016-08-03', '300万元人民币', '河北省秦皇岛市海港区承秦出海路东侧1-4-137号1#楼二楼', '13722557755@139.com', '0335-3306328 更多号码', '土石方工程、房屋建筑工程、室内外装饰装修工程、市政道路工程的设计、施工;建筑工程用机械设备租赁;提供劳务服务**(依法须经批准的项目,经相关部门批准后方可开展经营活动)', 'https://www.qcc.com/firm_941ef081134195e915552606aef17c0d.html', '91130302MA07U4N55P', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 1036
https://www.qcc.com/firm_7a3f6f197d851f8a3260ef77c4c419b3.html
无锡市锡山交通石化有限公司 company is done.
['', '无锡市锡山交通石化有限公司', '', '', '', '', '171743969@qq.com', '0510-83502025', '', 'https://www.qcc.com/firm_7a3f6f197d851f8a3260ef77c4c419b3.html', '', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 1037
https://www.qcc.com/firm_f621ed2281ee1fbbd5ba56f98f0a7e0f.html
中海华通石油化工（大连）有限公司 company is 

"dic_company_detail" Single is Done
now index is: 1052
https://www.qcc.com/firm_9fa66e18475a2d82459bf2dd8fe5cb13.html
珠海市立维科技发展有限公司 company is done.
['广东省', '珠海市立维科技发展有限公司', '黄翊', '2013-06-07', '500万元人民币', '珠海市横琴新区宝华路6号105室-23787(集中办公区)', '暂无', '18934228128 更多号码', '章程记载的经营范围:电子产品、节能产品研发、微电子表面贴装材料、封装材料、新材料、金属、化工的研究、生产、开发;微电子表面贴装材料、封装材料、电子废旧物技术及技术咨询服务;商务服务、国内贸易。(依法须经批准的项目,经相关部门批准后方可开展经营活动)', 'https://www.qcc.com/firm_9fa66e18475a2d82459bf2dd8fe5cb13.html', '914404000718542149', 1]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 1053
https://www.qcc.com/firm_f09dbcc836cf33f272624420aa4fcccb.html
广州柏乐昌化工有限公司 company is done.
['广东省', '广州柏乐昌化工有限公司', '黄斌', '2015-09-25', '500万元人民币', '广州高新技术产业开发区阅阳一街8号409房', '暂无', '13662883700 更多号码', '建材、装饰材料批发;商品批发贸易(许可审批类商品除外);橡胶制品批发;玻璃钢材料批发;玻璃钢制品批发;化工产品批发(危险化学品除外);合成纤维批发;货物进出口(专营专控商品除外);包装材料的销售;树脂及树脂制品批发;石油制品批发(成品油、危险化学品除外);润滑油批发;燃料油销售(不含成品油)', 'https://www.qcc.com/firm_f09dbcc836cf33f272624420aa4fcccb.html', '91440116MA59A66G4

In [ ]:
['单位','往来单位','应付账款','客户']

In [29]:
error_list = ['大连润涂石油化工有限公司', '大连绵亘商贸有限公司', '大连棉亘贸易有限公司', '杭州阡通信息科技有限公司', '陕西朗恒律师事务所', \
              '中国石化销售有限公司浙江嘉兴石油分公司（作废）']

with open(r'企查查无结果企业.txt','a',encoding='utf8') as w:
    for e in error_list:
        w.write(e+'\n')

In [54]:
# 对一些企业名字不规范的企业进行重新匹配id，如果直接用数据库的名字就不用这个cell重新匹配了

from DBUtils.PooledDB import PooledDB
import pymysql

_dbservice_pool = None

class MysqlPool(object): #设置数据库连接池和初始化    
    """
        MYSQL数据库对象，负责产生数据库连接 , 此类中的连接采用连接池实现
        获取连接对象：conn = Mysql.get_connection()
        释放连接对象;conn.close()或del conn
    """
    
    def __init__(self,mincached=5, maxcached=14,
                maxconnections=244, blocking=True, maxshared=0):
        """
            生成MySQL数据库连接池
        :param mincached: 最少的空闲连接数，如果空闲连接数小于这个数，pool会创建一个新的连接
        :param maxcached: 最大的空闲连接数，如果空闲连接数大于这个数，pool会关闭空闲连接
        :param maxconnections: 最大的连接数
        :param blocking: 当连接数达到最大的连接数时，在请求连接的时候，如果这个值是True，请求连接的程序会一直等待，
                          直到当前连接数小于最大连接数，如果这个值是False，会报错，
        :param maxshared: 当连接数达到这个数，新请求的连接会分享已经分配出去的连接
        """        
        db_config = {
            "host": '39.105.9.20',
            "port": 3306,
            "user": 'root',
            "passwd": 'bigdata_oil',
            "db": 'cxd_data',
            "charset": 'utf8'
        }
        self.pool = PooledDB(pymysql, mincached=mincached, maxcached=maxcached, maxconnections=maxconnections,
                             blocking=blocking, maxshared=maxshared, **db_config)

    def get_connection(self):
        return self.pool.connection()

    def close(self):
        self.pool.close()

    def __del__(self):
        self.close()

# 获取mysql数据库服务器的连接
def get_dbservice_mysql_conn(): #实例化连接池类，方便调用
    """
    :return: Object  MySQL Connection
    """
    global _dbservice_pool
    if not _dbservice_pool or not isinstance(_dbservice_pool, MysqlPool):
        _dbservice_pool = MysqlPool()
    return _dbservice_pool.get_connection()

connection = get_dbservice_mysql_conn()

def get_cookies():
    f= open('qichacha_cookies.txt')
    data_json = f.read()
    data_handle = json.loads(data_json)
    return data_handle

def get_company_info(name, cookies): 
    #name = '金钻石油机械股份有限公司'

    url_search = 'https://www.qichacha.com/search?key=%s' %name #目标网址
    browser = webdriver.Chrome()
    url_l= 'https://www.qichacha.com/user_login' #这一步特别重要，必须先打开一个目标网址对应的界面
    browser.get(url_l) 
    for cookie_item in cookies:
        #print({'name': cookie_item, 'value': cookies[cookie_item]})
        browser.add_cookie(cookie_item)
    browser.get(url_search)

    try:
        num = browser.find_element_by_xpath('//*[@id="countOld"]/span').text
    except Exception as e:
        print('基本被封，请手动查询')
        print(str(e))
        #return None

    if num.strip() == '0':
        print('无此企业', name)
        return 0
    else:
        time.sleep(1)
        search_url = browser.find_element_by_xpath('//*[@id="search-result"]/tr[1]/td[3]/a').get_attribute("href") 
        url_c = 'https://www.qichacha.com%s' %'/cbase'+search_url[29:-5]
        url_company = browser.get(url_c) 
        time.sleep(1)
        button = browser.find_element_by_xpath('//*[@id="firstepdadModal"]/div/div/div[2]/button')
        browser.execute_script("arguments[0].click();", button)
        try:
            time.sleep(1)
            company_title = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[1]/h1').text #企业名
            browser.quit()
            return company_title
        except Exception as e:
            print(repr(e))
            print('企业网页数据提取出现问题')
            return 0
        
def update_sql(name_sql, name_web): #更新不规范企业名字的id
    #name_sql数据库里的企业名称，name_web企查查搜到的企业名称
#     connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
    connection = get_dbservice_mysql_conn()
    sql_s = "SELECT `id`,`com_group`,`name` FROM `dic_company` WHERE `name` = '%s'" %name_web
    with connection.cursor() as cursor:
        cursor.execute(sql_s)
        result = cursor.fetchone()
    if result:
        with connection.cursor() as cursor:
            sql_up1 = "update `financial_exchange` set `Xf_company_id`=%s, `Xf_com_group`=%s where `Xf_company_name` = '%s'"\
            %(result[0],result[1],name_sql)
            cursor.execute(sql_up1)
#         with connection.cursor() as cursor:
            sql_up2 = "update `financial_exchange` set `Gf_company_id`=%s, `Gf_com_group`=%s where `Gf_company_name` = '%s'"\
            %(result[0],result[1],name_sql)
            cursor.execute(sql_up2)
        connection.commit()
        connection.close()
        print('上传完成{0}，企查查查询企业为{1}'.format(name_sql, name_web))
        return 0
    else:
        print('问题企业',name_sql)
        return 1
                
        

err_l = []
with open(r'需录入企查查企业_test.txt','r',encoding='utf8') as r:
    rows=r.readlines()
for l in rows:
    err_l.append(l.split('\n')[0])
    
print(len(err_l))
error_list = [i for i in err_l if len(i)>5]
print(len(error_list))

cookies = get_cookies()
error_coms = []
for index in range(460,len(error_list)):
    print('now index is:',index)        
    name_sql = error_list[index]
    print('目前不规范企业原名：', name_sql)
    name_web = get_company_info(name_sql, cookies)
    print("name_web:",name_web)
    if name_web == 0:
        error_coms.append(name_sql)
        continue
    error_judge = update_sql(name_sql, name_web)
    if error_judge == 1:
        error_coms.append(name_sql)
        print("目前无效企业名：",error_coms)
        continue
    time.sleep(random.uniform(54,147.7))
print(error_coms)

777
755
now index is: 460
目前不规范企业原名： 云南老猴科技有限公司
name_web: 云南老猴科技有限公司大理分公司
上传完成云南老猴科技有限公司，企查查查询企业为云南老猴科技有限公司大理分公司
now index is: 461
目前不规范企业原名： 池州市江南市政公用工程有限公司??
name_web: 池州市江南市政公用工程有限公司
上传完成池州市江南市政公用工程有限公司??，企查查查询企业为池州市江南市政公用工程有限公司
now index is: 462
目前不规范企业原名： 山东山岳化工有限公司
name_web: 山东山岳地坪工程有限公司
上传完成山东山岳化工有限公司，企查查查询企业为山东山岳地坪工程有限公司
now index is: 463
目前不规范企业原名： 辽宁省瓦房店市祝华办事处杨树房村
name_web: 瓦房店冰冰仓储有限公司
上传完成辽宁省瓦房店市祝华办事处杨树房村，企查查查询企业为瓦房店冰冰仓储有限公司
now index is: 464
目前不规范企业原名：  瓦房店中德瑞孚轴承制造有限公司
name_web: 瓦房店中德瑞孚轴承制造有限公司
上传完成 瓦房店中德瑞孚轴承制造有限公司，企查查查询企业为瓦房店中德瑞孚轴承制造有限公司
now index is: 465
目前不规范企业原名： 贵州川江建设工程有限公司望谟分公司
name_web: 贵州川江建设工程有限公司望谟分公司
上传完成贵州川江建设工程有限公司望谟分公司，企查查查询企业为贵州川江建设工程有限公司望谟分公司
now index is: 466
目前不规范企业原名： 遂昌县焦滩加油站
name_web: 遂昌县焦滩加油点
上传完成遂昌县焦滩加油站，企查查查询企业为遂昌县焦滩加油点
now index is: 467
目前不规范企业原名： 鄂尔多斯斯市瑞邦达经贸有限公司
name_web: 鄂尔多斯市瑞邦达经贸有限公司
上传完成鄂尔多斯斯市瑞邦达经贸有限公司，企查查查询企业为鄂尔多斯市瑞邦达经贸有限公司
now index is: 468
目前不规范企业原名： 苏陈建设集团有限公司
name_web: 江苏陈氏铧泰建设集团有限公司
上传完成苏陈建设集团有限公司，企查查查询企业为江苏陈氏铧泰建设集团有限公司
now index is: 469

name_web: 易大宗（北京）供应链管理有限公司
上传完成易大宗（北京）供应链管理有限公司		，企查查查询企业为易大宗（北京）供应链管理有限公司
now index is: 535
目前不规范企业原名： 大连华富石油化工（大连）有限公司
name_web: 中油华富石油化工（大连）有限公司
上传完成大连华富石油化工（大连）有限公司，企查查查询企业为中油华富石油化工（大连）有限公司
now index is: 536
目前不规范企业原名： 中铁五局集团第一工程有限公司
name_web: 中铁二十五局集团第一工程有限公司
上传完成中铁五局集团第一工程有限公司，企查查查询企业为中铁二十五局集团第一工程有限公司
now index is: 537
目前不规范企业原名： 全球通管道有限公司
name_web: 湖南全球通管道有限公司
上传完成全球通管道有限公司，企查查查询企业为湖南全球通管道有限公司
now index is: 538
目前不规范企业原名： 湖南大成景观有限公司
name_web: 湖南筑融丰建设有限公司
上传完成湖南大成景观有限公司，企查查查询企业为湖南筑融丰建设有限公司
now index is: 539
目前不规范企业原名： 湖南省金尚工程建设有限公司	
name_web: 湖南省金尚工程建设有限公司
上传完成湖南省金尚工程建设有限公司	，企查查查询企业为湖南省金尚工程建设有限公司
now index is: 540
目前不规范企业原名： 汇通天下石油化工（大连） 有限公司
name_web: 汇通天下石油化工（大连）有限公司
上传完成汇通天下石油化工（大连） 有限公司，企查查查询企业为汇通天下石油化工（大连）有限公司
now index is: 541
目前不规范企业原名： 南宁市志德港埠有限公
name_web: 南宁市志德港埠有限公司
上传完成南宁市志德港埠有限公，企查查查询企业为南宁市志德港埠有限公司
now index is: 542
目前不规范企业原名： 中物国际供应链股份有限公司
name_web: 中物国际供应链集团股份有限公司
上传完成中物国际供应链股份有限公司，企查查查询企业为中物国际供应链集团股份有限公司
now index is: 543
目前不规范企业原名： 能源石化（大连）有限公司
name_web: 大连金海船务有限公

上传完成乌兰察布市浩通能源有限责，企查查查询企业为
now index is: 612
目前不规范企业原名： 淄博宝贝之家服务有限公司
name_web: 
上传完成淄博宝贝之家服务有限公司，企查查查询企业为
now index is: 613
目前不规范企业原名： 威海市博文贸易有限公司
name_web: 威海博文贸易有限公司
上传完成威海市博文贸易有限公司，企查查查询企业为威海博文贸易有限公司
now index is: 614
目前不规范企业原名： 四川迪拓建设工程有限公司
name_web: 
上传完成四川迪拓建设工程有限公司，企查查查询企业为
now index is: 615
目前不规范企业原名： 鑫峦贸易(上海)有限公司
name_web: 鑫峦贸易（上海）有限公司
上传完成鑫峦贸易(上海)有限公司，企查查查询企业为鑫峦贸易（上海）有限公司
now index is: 616
目前不规范企业原名： 山东泰森日盛家居科技有限公司
name_web: 
上传完成山东泰森日盛家居科技有限公司，企查查查询企业为
now index is: 617
目前不规范企业原名： 中外运-敦豪国际航空快件有限公司珠海分公司
name_web: 
上传完成中外运-敦豪国际航空快件有限公司珠海分公司，企查查查询企业为
now index is: 618
目前不规范企业原名： 曼宁家屋面系统(佛山)有限公司
name_web: 
上传完成曼宁家屋面系统(佛山)有限公司，企查查查询企业为
now index is: 619
目前不规范企业原名： 延长壳牌(广东)石油有限公司
name_web: 
上传完成延长壳牌(广东)石油有限公司，企查查查询企业为
now index is: 620
目前不规范企业原名： 鄂尔多斯市路泰公路有限责任公司
name_web: 
上传完成鄂尔多斯市路泰公路有限责任公司，企查查查询企业为
now index is: 621
目前不规范企业原名： 华信国际（沈阳）石油有限公司
name_web: 华贸（辽宁）能源有限公司
上传完成华信国际（沈阳）石油有限公司，企查查查询企业为华贸（辽宁）能源有限公司
now index is: 622
目前不规范企业原名： 博兴乔博加油站
name_web: 博兴县乔博加油站
上传完成博兴乔博加油站，企查查查询企业为

name_web: 
上传完成青海振通建筑基础有限公司，企查查查询企业为
now index is: 693
目前不规范企业原名：  葫芦岛凌云集团农药化工有限公司
name_web: 
上传完成 葫芦岛凌云集团农药化工有限公司，企查查查询企业为
now index is: 694
目前不规范企业原名： 沈阳源辰石油化工有限公司
name_web: 辽宁国科能源有限公司
上传完成沈阳源辰石油化工有限公司，企查查查询企业为辽宁国科能源有限公司
now index is: 695
目前不规范企业原名： 中洁燃(浙江自贸区)能源有限公司
name_web: 中洁燃（浙江自贸区）能源有限公司
上传完成中洁燃(浙江自贸区)能源有限公司，企查查查询企业为中洁燃（浙江自贸区）能源有限公司
now index is: 696
目前不规范企业原名： 北票市富盛实业有限公司
name_web: 北票富盛实业有限公司
上传完成北票市富盛实业有限公司，企查查查询企业为北票富盛实业有限公司
now index is: 697
目前不规范企业原名： 云南丰博工程建设有限公司
name_web: 云南丰博工程建设有限公司罗平分公司
上传完成云南丰博工程建设有限公司，企查查查询企业为云南丰博工程建设有限公司罗平分公司
now index is: 698
目前不规范企业原名： 中油兴润能源(大连)有限公司
name_web: 中油兴润能源（大连）有限公司
上传完成中油兴润能源(大连)有限公司，企查查查询企业为中油兴润能源（大连）有限公司
now index is: 699
目前不规范企业原名： 中丝(大连)石化有限公司
name_web: 中丝（大连）石化有限公司
上传完成中丝(大连)石化有限公司，企查查查询企业为中丝（大连）石化有限公司
now index is: 700
目前不规范企业原名： 中丝（大连)石化有限公司
name_web: 中丝（大连）石化有限公司
上传完成中丝（大连)石化有限公司，企查查查询企业为中丝（大连）石化有限公司
now index is: 701
目前不规范企业原名： 中油中铝(大连)石油化工有限公司
name_web: 中油中铝（大连）石油化工有限公司
上传完成中油中铝(大连)石油化工有限公司，企查查查询企业为中油中铝（大连）石油化工有限公司
now index is

In [19]:
#修改in_total表无id企业名

#com_origin：404企业原名
#com_search: 正确企业名

from DBUtils.PooledDB import PooledDB
import pymysql

_dbservice_pool = None

class MysqlPool(object): #设置数据库连接池和初始化    
    """
        MYSQL数据库对象，负责产生数据库连接 , 此类中的连接采用连接池实现
        获取连接对象：conn = Mysql.get_connection()
        释放连接对象;conn.close()或del conn
    """
    
    def __init__(self,mincached=5, maxcached=14,
                maxconnections=244, blocking=True, maxshared=0):
        """
            生成MySQL数据库连接池
        :param mincached: 最少的空闲连接数，如果空闲连接数小于这个数，pool会创建一个新的连接
        :param maxcached: 最大的空闲连接数，如果空闲连接数大于这个数，pool会关闭空闲连接
        :param maxconnections: 最大的连接数
        :param blocking: 当连接数达到最大的连接数时，在请求连接的时候，如果这个值是True，请求连接的程序会一直等待，
                          直到当前连接数小于最大连接数，如果这个值是False，会报错，
        :param maxshared: 当连接数达到这个数，新请求的连接会分享已经分配出去的连接
        """        
        db_config = {
            "host": '39.105.9.20',
            "port": 3306,
            "user": 'root',
            "passwd": 'bigdata_oil',
            "db": 'cxd_test',
            "charset": 'utf8'
        }
        self.pool = PooledDB(pymysql, mincached=mincached, maxcached=maxcached, maxconnections=maxconnections,
                             blocking=blocking, maxshared=maxshared, **db_config)

    def get_connection(self):
        return self.pool.connection()

    def close(self):
        self.pool.close()

    def __del__(self):
        self.close()

# 获取mysql数据库服务器的连接
def get_dbservice_mysql_conn(): #实例化连接池类，方便调用
    """
    :return: Object  MySQL Connection
    """
    global _dbservice_pool
    if not _dbservice_pool or not isinstance(_dbservice_pool, MysqlPool):
        _dbservice_pool = MysqlPool()
    return _dbservice_pool.get_connection()

conn = get_dbservice_mysql_conn()



# conn = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
# conn = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_test',charset = 'utf8')

com_search = []

def update_in_total_name(com_origin, com_correct):
    connection = conn
    with connection.cursor() as cursor:
        sql = "update `in_total` set `com_name`='{0}' where com_name='{1}'".format(com_correct,com_origin)
        cursor.execute(sql)
    connection.commit()
    connection.close()

# com_search.append(name)

connection = conn
sql = "SELECT `com_id`,`com_name` FROM `in_total` WHERE `com_id` is null"
with connection.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()
com_origin = [com[1] for com in result]

for com_name in com_origin:
    if com_name.endswith('科目余额表'):
        name = com_name[:-5]
        update_in_total_name(com_name, name)
    elif com_name.endswith('序时账'):
        name = com_name[:-3]
        update_in_total_name(com_name, name)
    elif com_name.endswith('19'):
        name = com_name[:-2]
        update_in_total_name(com_name, name)
    else: 
        name = com_name
    com_search.append(name)
            
# def get_com_id(name):
#     connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')

# #     name = '大连大昌石油化工有限公司'

#     sql = "SELECT `id`,`name` FROM `dic_company` WHERE `name` ='%s'" %name
#     with connection.cursor() as cursor:
#         cursor.execute(sql)
#         result = cursor.fetchone()
#     return result[0]

def update_in_total_id(com_name):
    print(com_name)
    connection = conn
    with connection.cursor() as cursor:
        sql = "SELECT `id`,`name` FROM `dic_company` WHERE `name` ='%s'" %com_name
        cursor.execute(sql)
        result = cursor.fetchone()
    print(result)
    com_id = result[0]
    with connection.cursor() as cursor:
        sql = "update `in_total` set `com_id`='{0}' where com_name='{1}'".format(com_id,com_name)
        print(sql)
        cursor.execute(sql)
    connection.commit()
    connection.close()

for com in com_search:
    update_in_total_id(com)
print('Done')    

中油永盛石化（大连）有限公司
(2438, '中油永盛石化（大连）有限公司')
update `in_total` set `com_id`='2438' where com_name='中油永盛石化（大连）有限公司'
中油满世联合石化（大连）有限公司


InterfaceError: (0, '')

In [16]:
#in_total表无id企业（404页面）

import pymysql

# connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_test',charset = 'utf8')

sql = "SELECT `com_id`,`com_name` FROM `in_total` WHERE `com_id` is null"
with connection.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()
print(result)

com_search = []
# for row in result:
#     com_name = row[1]
#     if com_name.endswith('科目余额表'):
#         name = com_name[:-5]
#     elif com_name.endswith('序时账'):
#         name = com_name[:-3]
#     elif com_name.endswith('19'):
#         name = com_name[:-2]
#     else: 
#         name = com_name
#     com_search.append(name)

# com_search    

((None, '中油中铝（大连）石油化工有限公司'), (None, '中油智胜石油化工（大连）有限公司'), (None, '中油永盛石化（大连）有限公司'), (None, '中油满世联合石化（大连）有限公司'), (None, '中铁三局集团（大连）石化有限公司'), (None, '大连中油鑫昊石油化工有限公司'), (None, '大连中物石化有限公司'), (None, '大连中航通用贸易有限公司'), (None, '大连中资石油化工有限公司'), (None, '大连北方化学工业有限公司'), (None, '大连华油润达石油化工有限公司'), (None, '大连国合经贸有限公司'), (None, '大连天贻国际贸易有限公司'), (None, '大连欣恒石油化工有限公司'), (None, '大连欣龙石油化工有限公司'), (None, '大连海益石油化工有限公司'), (None, '大连环通商贸有限公司'), (None, '大连聚生源石油化工有限公司'), (None, '大连蓝达科技有限公司'), (None, '大连裕泰石油化工有限公司'), (None, '大连金润二当家石油化工有限公司'), (None, '大连鸿润达石油化工有限公司'), (None, '大连鸿淼石油化工有限公司'), (None, '大连鼎新商贸有限公司'), (None, '山恩能源（大连）有限公司'), (None, '山东中油胜利石油销售有限公司'), (None, '中国工商银行股份有限公司北京市分行'), (None, '山东中油胜利石化有限公司'), (None, '吉林市昌邑区赛菲尔商务酒店'), (None, '东明中油燃料石化有限公司'), (None, '长沙金鼠服饰有限公司'), (None, '去哪儿网（天津）国际旅行社有限公司'), (None, '北京富澳德服装服饰有限责任公司'), (None, '北京中油北油化工有限公司'), (None, '新乡市平原示范区祥瑞商务温泉快捷酒店'), (None, '大同贵宾楼酒店有限公司'), (None, '四川亨源石化有限责任公司简阳分公司'), (None, '营口市站前区驿家旅馆'), (None, '泗阳泽熙商贸有限公司'), (None, '江苏金翔石油化工有限公司'), (N

In [36]:
cookies = get_cookies()
name_test = get_company_info('北方华锦化学工业',cookies)
print(name_test)

connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')

sql = "SELECT `id`,`name` FROM `dic_company` WHERE `name` = '%s'" %name_test
with connection.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchone()
print(result)

北方华锦化学工业集团有限公司
(4390, '北方华锦化学工业集团有限公司')


In [45]:
connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
with connection.cursor() as cursor:
    sql_up1 = "update `financial_exchange` set `Xf_company_id`=%s, `Xf_com_group`=%s where `Xf_company_name` = '%s'" %(4390,0,'北方华锦化学工业')
    print(sql_up1)
    cursor.execute(sql_up1)
# with connection.cursor() as cursor:
#     sql_up2 = "update `financial_exchange` set `Gf_company_id`=%s, `Gf_com_group`=%s where `Gf_company_name` = %s"
#     cursor.execute(sql_up2,[result[0],result[1],name_sql])
connection.commit()
connection.close()

update `financial_exchange` set `Xf_company_id`=4390, `Xf_com_group`=0 where `Xf_company_name` = '北方华锦化学工业'


In [29]:
f= open('qichacha_cookies.txt')
data_json = f.read()
data_handle = json.loads(data_json)
print(type(data_handle))
print(data_handle)

<class 'list'>
[{'domain': '.qichacha.com', 'expiry': 1607396978, 'httpOnly': False, 'name': 'Hm_lvt_3456bee468c83cc63fb5147f119f1075', 'path': '/', 'secure': False, 'value': '1575860979'}, {'domain': '.qichacha.com', 'expiry': 1591585778, 'httpOnly': False, 'name': 'UM_distinctid', 'path': '/', 'secure': False, 'value': '16ee8a242df12d-0f9bfa08c6d415-2393f61-1fa400-16ee8a242e0af3'}, {'domain': '.qichacha.com', 'httpOnly': False, 'name': 'Hm_lpvt_3456bee468c83cc63fb5147f119f1075', 'path': '/', 'secure': False, 'value': '1575860979'}, {'domain': '.qichacha.com', 'expiry': 1607396978, 'httpOnly': False, 'name': 'zg_did', 'path': '/', 'secure': False, 'value': '%7B%22did%22%3A%20%2216ee8a241e3a46-0cf961470bb314-2393f61-1fa400-16ee8a241e492f%22%7D'}, {'domain': '.qichacha.com', 'expiry': 1607396978, 'httpOnly': False, 'name': 'zg_de1d1a35bfa24ce29bbf2c7eb17e6c4f', 'path': '/', 'secure': False, 'value': '%7B%22sid%22%3A%201575860978151%2C%22updated%22%3A%201575860978151%2C%22info%22%3A%2015

In [11]:
url = 'https://www.qichacha.com/cbase_a700673c1a64195944db0092fe0f78bb.html' #目标网址
browser = webdriver.Chrome()
url_l= 'https://www.qichacha.com/user_login' #这一步特别重要，必须先打开一个目标网址对应的界面
browser.get(url_l) 
for cookie_item in data_handle:
    #print({'name': cookie_item, 'value': cookies[cookie_item]})
    browser.add_cookie(data_handle)
browser.get(url)

In [8]:
q = QichachaSelenium()
cookies = q.get_cookies()

name = '鸿祥源石化(大连）有限公司'

url_search = 'https://www.qichacha.com/search?key=%s' %name #目标网址
browser = webdriver.Chrome()
url_l= 'https://www.qichacha.com/user_login' #这一步特别重要，必须先打开一个目标网址对应的界面
browser.get(url_l) 
for cookie_item in cookies:
    #print({'name': cookie_item, 'value': cookies[cookie_item]})
    browser.add_cookie(cookie_item)
browser.get(url_search)

try:
    num = browser.find_element_by_xpath('//*[@id="countOld"]/span').text
except Exception as e:
    print('基本被封，请手动查询')
    print(str(e))
    #return None

if num.strip() == '0':
    print('无此企业', name)
#     return 0
else:
    time.sleep(1)
    search_url = browser.find_element_by_xpath('//*[@id="search-result"]/tr[1]/td[3]/a').get_attribute("href") 
    url_c = 'https://www.qichacha.com%s' %'/cbase'+search_url[29:-5]
    #print(url_c)
    #com_web_q = 'https://www.qichacha.com' + search_url #企查查的企业主页
    url_company = browser.get(url_c) 
    time.sleep(1)
    button = browser.find_element_by_xpath('//*[@id="firstepdadModal"]/div/div/div[2]/button')
    browser.execute_script("arguments[0].click();", button)
    time.sleep(1)
    #table =  browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody').text
    #print(table)
    company_title = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[1]/h1').text #企业名
    company_lead = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[2]/div/div/div[2]/a[1]/h2').text #法人
    company_tel = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[3]/div[1]/span[1]/span[2]').text.strip()
    company_email = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[3]/div[2]/span[1]/span[2]').text.strip().split(' ')[0]
    #print(company_lead, company_tel, company_email)

    found_cash = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[2]/td[2]').text.strip() #注册资本
    found_date = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[6]').text.strip() #成立日期
    company_loc = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[8]/td[2]').text.split('\n')[0] #企业地址'
    company_cover = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[9]/td[2]').text.strip() # 经营范围
    company_tax_num = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[4]/td[2]').text.strip() #'纳税人识别号'
    company_prov = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[6]/td[6]').text.strip()  #所属地区
    #print('公司名称：%s' %company_title)
    try:
        info_0 = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[4]').text.strip() #经营状态
    except:
        info_0 = None
    if '在业' in info_0:
        info = 1
    elif '存续' in info_0:
        info = 2
    elif '注销' in info_0:
        info = 10
    elif '吊销' in info_0:
        info = 9
    elif '筹建' in info_0:
        info = 3
    elif '清算' in info_0:
        info = 4
    elif '迁入' in info_0:
        info = 5
    elif '迁出' in info_0:
        info = 6
    elif '停业' in info_0:
        info = 7
    elif '撤销' in info_0:
        info = 8
    elif '开业' in info_0:
        info = 1
    else:
        info = None
    msg = [company_prov, company_title, company_lead, found_date, found_cash, company_loc, company_email, company_tel,\
           company_cover, search_url, company_tax_num, info]
    print('%s company is done.' %name)
    print(msg)
    
    #[company_prov, name, company_lead, found_date, found_cash, company_loc, company_email, company_tel,\
                       #company_cover, search_url, company_tax_num, info]

鸿祥源石化(大连）有限公司 company is done.
['辽宁省', '鸿祥源石化（大连）有限公司', '王丽丽', '31141870-3', '存续（在营、开业、在册）', '辽宁省大连长兴岛经济区何屯村(原何屯小学) 查看地图 附近企业', '13339337343@qq.com 更多邮箱', '0411-82518632', '成品油(仅限汽油、煤油(含3号喷气燃料)、柴油[闭杯闪点≤60℃])、石油原油、石脑油、石油醚、煤焦油、溶剂油[闭杯闪点≤60℃]、苯乙烯[稳定的]、苯酚、苯、粗苯、重质苯、乙酸乙酯、乙酸正丁酯、1,2-二甲苯、1,3-二甲苯、1,4-二甲苯、二甲苯异构体混合物、乙苯、1,2,3-三甲基苯、1,2,4-三甲基苯、1,3,5-三甲基苯、1,2,4,5-四甲苯、乙烯、丙烯、1-丁烯、2-丁烯、1,3-丁二烯[稳定的]、异丁烯、2-戊烯、环戊烯、2-甲基-2-丁烯、3-甲基-1-丁烯、甲烷、乙烷、丙烷、正丁烷、异丁烷、环丁烷、正戊烷、环戊烷、2-甲基丁烷、2-甲基戊烷、正己烷、异辛烷、1,2-环氧丙烷、环氧乙烷、乙炔、甲基叔丁基醚、甲醇、乙醇[无水]、正丁醇、环己酮、氢氧化钠、煤焦沥青、硝化沥青、三聚丙烯、四聚丙烯、蒽油乳膏、蒽油乳剂、含易燃溶剂的合成树脂、油漆、辅助材料、涂料等制品:[碳四、碳五、碳九、混合芳烃、重芳烃、轻芳烃、稳定轻烃、混合苯、混合二甲苯、重烯烃、杂醇、轻质燃料油、轻质油、汽油调和组分、洗油(81)][闭杯闪点≤60℃](2828)的无储存经营,成品油(仅限柴油[闭杯闪点>60℃])、燃料油[闭杯闪点>60℃](以上均不含危险化学品)、沥青、润滑油、钢材、木材、煤炭、仪器仪表、电线电缆、矿产品、阀门、办公用品、汽车零部件、计算机硬件、建材、五金产品、化工产品销售,经营液化石油气、天然气(以上仅限城镇燃气经营),经济信息咨询(不得从事教育培训及办学),计算机软件开发,经营广告业务,国内一般贸易,货物及技术进出口。(依法须经批准的项目,经相关部门批准后方可开展经营活动)***(依法须经批准的项目,经相关部门批准后方可开展经营活动。)', 'https://www.qichacha.com/firm_024fd7f9b70aaf36bbe2f3c6dfaaaf25.html', '91210244311418703E'

In [50]:
company_loc.split(' ')[0]

'准格尔旗暖水乡暖水村'

In [10]:
'13339337343@qq.com 更多邮箱'.split(' ')[0]

'13339337343@qq.com'

In [ ]:
//*[@id="Cominfo"]/table/tbody/tr[3]/td[2]
//*[@id="Cominfo"]/table/tbody/tr[3]/td[2]